In [9]:
import re, string, os, glob
import pandas as pd
from bs4 import BeautifulSoup
import requests
import GetOldTweets3 as got 
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import CountVectorizer
from stop_words import get_stop_words
from ipywidgets import widgets
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
plt.style.use('ggplot')

global KEYWORDS
global DF
global ANALYSIS
global TEXT_SOURCE
global WORDS
global TOPICS

In [2]:
def get_text_from_url(url):
    if url:
        try:
            soup = BeautifulSoup(requests.get(url).text, 'html.parser')
            return " ".join([p.get_text().replace(u'\xa0', u' ') for p in soup.find_all('p')])
        except:
            return ""
    else:
        return ""

def get_top_words(model, feature_names, n_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        topics[topic_idx] = " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
    return topics

def get_topics(texts, 
               n_components=20, 
               n_top_words = 20, 
               n_features=int(1e12),
               keywords=[],
               topic_model='NMF'
              ):
    vect = CountVectorizer(max_df=0.8, 
                           min_df=1,
                           max_features=n_features, 
                           stop_words=get_stop_words('de') + keywords)
    if topic_model == 'LDA':
        model = LatentDirichletAllocation(n_components=n_components, max_iter=200,
                                learning_method='online',
                                random_state=0)
    elif topic_model == 'NMF':
        model = NMF(n_components=n_components, random_state=0)
    tf = vect.fit_transform(texts)
    model.fit(tf)
    topic_assignments = model.transform(tf)
    tf_feature_names = vect.get_feature_names()
    topics_dict = get_top_words(model, tf_feature_names, n_top_words)
    
    # sort topics by occurrence
    topic_loadings = topic_assignments.sum(axis=0).argsort()[:-1:]
    topic_assignments = topic_assignments[:,topic_loadings]
    topics = [topics_dict[topic_idx] for topic_idx in topic_loadings]
    return topic_assignments.argmax(axis=1), topics

def get_top_word_counts(texts, n_top_words, keywords):
    vect = CountVectorizer(max_df=1., 
                           min_df=1,
                           max_features=n_top_words, 
                           stop_words=get_stop_words('de') + keywords)
    wordcounts = vect.fit_transform(texts)
    return wordcounts, vect.get_feature_names()

# Twitter Analyse mit Topic Models (oder ohne)

## Tweets Runterladen

Lade Tweets aus deutschsprachigem Raum für den angegebenen Zeitraum, die die angegebenen keywords enthalten.

Wenn maxTweets auf 0 gesetzt ist, wird versucht alle zu holen. Das kann dauern, und manchmal ist man danach gedrosselt. 

Klicke ``Run Interact`` button zum ausführen.

In [3]:
@widgets.interact_manual(maxTweets=(0, 100))
def get_tweets(keywords='covid klopapier', 
               start=widgets.DatePicker(value=pd.to_datetime('2020-04-10')), 
               stop=widgets.DatePicker(value=pd.to_datetime('2020-04-16')), 
               maxTweets=10):
    print(f'Fetching tweets for keywords: {keywords}')
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(keywords)\
                                             .setSince(start.strftime('%Y-%m-%d'))\
                                             .setUntil(stop.strftime('%Y-%m-%d'))\
                                             .setMaxTweets(maxTweets)\
                                             .setLang('de')\
                                             .setNear('Berlin, Germany')\
                                             .setWithin('1000km') 
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    tweet_dicts = []
    for tweet in tweets:
        tweet_dict = tweet.__dict__ 
        tweet_dict['url_text'] = get_text_from_url(tweet.urls)
        tweet_dicts.append(tweet_dict)
        
    print(f'Found {len(tweets)} tweets for keywords: {keywords}')
    global DF
    DF = pd.DataFrame(tweet_dicts).set_index('date')
    global KEYWORDS
    KEYWORDS = re.split("[" + string.punctuation + " \t\n]+", keywords)

interactive(children=(Text(value='covid klopapier', description='keywords'), DatePicker(value=Timestamp('2020-…

## Tweets Analysieren

- ``text_source``: Wähle, ob Text aus tweets oder den in tweets verlinkten Artikeln analysiert werden soll. 
- ``analysis``: Wähle ob Häufigkeit einzelner Worte oder Topics analysiert werden.
- ``n_top_words``: wie viele Worte berücksichtigt werden oder bei Topic Analysis, wieviele Wòrter angezeigt werden
- ``topic_model``: Art des Topic Models (Latent Dirichlet Allocation oder Nonnegative Matrix Factorization) - wird ignoriert, wnen ``analysis`` auf ``wordcount`` gesetzt ist
- ``num_topics``: Wie viele Topics extrahiert werden soll.

Klicke ``Run Interact`` button zum ausführen.

In [3]:
@widgets.interact_manual(num_topics=(2, 20),
                         text_source=['tweets','urls in tweets'],
                         analysis=['topic model', 'wordcounts'],
                         n_top_words = (5, 20),
                         topic_model=['LDA','NMF']
                        )
def plot_tweets(
         text_source='tweets',
         analysis='topic model',
         n_top_words=5,
         topic_model='NMF',
         num_topics=10    
        ):
    global DF
    global TOPICS
    global ANALYSIS 
    ANALYSIS = analysis
    global TEXT_SOURCE
    TEXT_SOURCE = text_source
    if len(DF) == 0:
        print("Get some tweets first.")
        return
    else:
        print(f"Analysing {len(DF)} {text_source} with {analysis}, exluding keywords: {KEYWORDS}")
        
        if text_source == 'tweets':
            text_col = 'text'
        elif text_source == 'urls in tweets':            
            text_col = 'url_text'
            
        if analysis == 'topic model':
            DF['topic'], TOPICS = get_topics(DF[text_col], 
                                                    n_components=num_topics, 
                                                    n_top_words=n_top_words,
                                                    keywords=KEYWORDS,
                                                    topic_model=topic_model
                                                  )
            pd.get_dummies(DF['topic']).resample('D').sum().plot(marker='o')
            plt.legend([f'Topic {idx}: {text[:10]} ...' for idx, text in enumerate(TOPICS)]);
            print("\n".join([f'Topic {idx}: {text}' for idx, text in enumerate(TOPICS)]))
        elif analysis == 'wordcounts':
            word_counts, words = get_top_word_counts(DF[text_col],  
                                                     n_top_words=n_top_words,
                                                    keywords=KEYWORDS)
            global TOP_WORDS
            TOP_WORDS = words
            pd.DataFrame(word_counts.toarray(), 
                         index=DF.index, 
                         columns=words).resample('D').sum().plot(marker='o')
            

interactive(children=(Dropdown(description='text_source', options=('tweets', 'urls in tweets'), value='tweets'…

## Tweets Anschauen

In [4]:
from IPython.html.widgets import interactive

try:
    df = DF.copy(deep=True)
except NameError:
    DF = pd.DataFrame([],columns=['username','to','text','topic','permalink'])
    df = DF.copy(deep=True)
    TEXT_SOURCE = 'tweets'
    ANALYSIS = 'wordcounts'
    TOP_WORDS = []

if TEXT_SOURCE == 'tweets':
    text_col = 'text'
elif TEXT_SOURCE == 'urls in tweets':
    text_col = 'url_text'

if ANALYSIS == 'wordcounts':
    items = ['All'] + sorted(TOP_WORDS)
elif ANALYSIS == 'topic model':
    items = ['All'] + [f'{idx}: {text[:20]} ...' for idx, text in enumerate(TOPICS)]
    
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)
    
def view(contains=''):
    if contains=='All': 
        print(f'Showing all {len(df)} tweets')
        display(df.loc[:,['username','to','text','permalink']].style.format({'permalink': make_clickable}))
    elif ANALYSIS == 'wordcounts':
        idx = df[text_col].str.lower().str.contains(contains.lower())
        print(f'Found {idx.sum()} tweets')
        display(df.loc[idx,['username','to','text','permalink']].style.format({'permalink': make_clickable}))
    elif ANALYSIS == 'topic model':
        idx = df['topic'] == int(contains[:contains.find(':')])
        print(f'Found {idx.sum()} tweets')
        display(df.loc[idx,['username','to','text','topic','permalink']].style.format({'permalink': make_clickable}))

w = widgets.Select(options=items)
interactive(view, contains=w)

interactive(children=(Select(description='contains', options=('All',), value='All'), Output()), _dom_classes=(…

In [14]:
tweet_files = glob.glob(os.path.join('twitterdata','*','*.json'))
df_ = pd.read_json(tweet_files[10])

ValueError: Trailing data

In [6]:
DF

,username,to,text,retweets,favorites,replies,id,permalink,author_id,formatted_date,hashtags,mentions,geo,urls,url_text
date,,,,,,,,,,,,,,,
2020-04-15 23:37:15+00:00,vv_viktoria,None,Die neue Therapie mit #Blutplasmaspende an der...,0,0,0,1250568904008708096,https://twitter.com/vv_viktoria/status/1250568...,445775804,Wed Apr 15 23:37:15 +0000 2020,#Blutplasmaspende #TogetherWeRise #covid,,,,
2020-04-15 22:58:38+00:00,DamarisSieglind,sofatiere,Sie unterliegen leider den finanziellen Zwänge...,0,2,0,1250559186217435142,https://twitter.com/DamarisSieglind/status/125...,937754666401755136,Wed Apr 15 22:58:38 +0000 2020,,,,,
2020-04-15 21:32:06+00:00,PeterHeidtFDP,c_lindner,"Aktuell sinken die Zahlen der Neuinfektionen, ...",1,1,0,1250537408409600000,https://twitter.com/PeterHeidtFDP/status/12505...,1858621434,Wed Apr 15 21:32:06 +0000 2020,,,,,
2020-04-15 21:01:39+00:00,Knippkopf,lange_tobias_hh,"Jetzt komm mir noch mit ""Es gab sogar Klopapie...",0,2,0,1250529746657689603,https://twitter.com/Knippkopf/status/125052974...,2305238089,Wed Apr 15 21:01:39 +0000 2020,,,,,
2020-04-15 20:30:04+00:00,JoergRMayer,SimonKohlbauer,"Was das mit rechts und links zu tun hat, ersch...",0,2,0,1250521798141194240,https://twitter.com/JoergRMayer/status/1250521...,3008773191,Wed Apr 15 20:30:04 +0000 2020,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-10 05:02:10+00:00,StefanKuster,BAG_OFSP_UFSP,Peinlicher geht nimmer @BAG_OFSP_UFSP Löffel e...,0,3,0,1248476341747331078,https://twitter.com/StefanKuster/status/124847...,292798962,Fri Apr 10 05:02:10 +0000 2020,#peinlich #Covid_19,@BAG_OFSP_UFSP,,,
2020-04-10 02:58:58+00:00,nice_pebbles,None,Guten Morgen und frohe #Ostern trotz (oder ger...,0,18,0,1248445340493262849,https://twitter.com/nice_pebbles/status/124844...,4219168216,Fri Apr 10 02:58:58 +0000 2020,#Ostern #COVID #stabil #sge #eintracht,@Eintracht,,,
2020-04-10 02:37:02+00:00,covid_watch,None,Coronavirus fordert Pflegedienste,1,1,0,1248439819417423872,https://twitter.com/covid_watch/status/1248439...,1240313872927653892,Fri Apr 10 02:37:02 +0000 2020,,,,http://dlvr.it/RTVzPB,"\n NÖN.at verwendet Cookies, um..."


In [1]:
from classifier import *
train_all()
df = get_bundestag_data()
df = score_texts(df)
df.to_csv("bundestagsprotokolle/bundestagsprotokolle_categorized.csv.gz",index=False,compression='gzip')

Loading manifesto/manifesto-Germany.csv
Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    3.0s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    3.0s finished


Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.8s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.4s finished


Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    5.1s remaining:    5.1s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    5.2s finished
